# Tabular Playgroud Series - December 2021

## 0. Modules, Data Read and dataframe dimension reduction

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from catboost import CatBoostClassifier

In [ ]:
#Read the data

#train data
train = pd.read_csv("../input/tabular-playground-series-dec-2021/train.csv")

#test data
test = pd.read_csv("../input/tabular-playground-series-dec-2021/test.csv")

#sample submission
submission = pd.read_csv("../input/tabular-playground-series-dec-2021/sample_submission.csv",index_col='Id')

In [ ]:
#reduce the size of dataframe. Credits to https://www.kaggle.com/c/tabular-playground-series-dec-2021/discussion/291844

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
#aply reduction to train dataframe

train_df = reduce_mem_usage(train)

In [ ]:
#aply reduction to test dataframe

test_df = reduce_mem_usage(test)

In [ ]:
#columns names

columnas = train_df.columns
train_df[columnas].dtypes

## 1. Exploratory Data Analisys

### 1.1 Continuos Variables

In [ ]:
#function for kde plot - Train and Test

def kde_plot(var):
    fig = plt.figure(figsize=(10,6))
    sns.kdeplot(data=train_df, x=var)
    sns.kdeplot(data=test_df, x=var)
    fig.legend(labels=['train','test'])
    plt.title(var)
    plt.show()

In [ ]:
#Plot all numerical variables

for i in columnas[1:11]:
    kde_plot(i)

In principle, no data drifting is observed between the corresponding variables train and test

### 1.2 Categorical Variables

In [ ]:
#function for kde plot - Train and Test

def count_plot(var):
    fig,axs = plt.subplots(ncols=2,figsize=(16,9))
    sns.countplot(x=var,data=train_df,ax=axs[0])
    sns.countplot(x=var,data=test_df,ax=axs[1])
    axs[0].title.set_text('Train')
    axs[1].title.set_text('Test')
    plt.show()

In [ ]:
#Plot all numerical variables

for i in columnas[12:-1]:
    count_plot(i)

### 1.3 Target

In [ ]:
fig = plt.figure(figsize=(10,6))
sns.countplot(x="Cover_Type",data=train_df)
plt.show()

## 2. Model

In [ ]:
train.columns

In [ ]:
x = train[train.columns[~train.columns.isin(['Id','Cover_Type'])]]
x.head()

In [ ]:
y = train['Cover_Type']
y.head()

In [ ]:
model = CatBoostClassifier()

In [ ]:
grid = {'learning_rate': [0.03, 0.1],
        'depth': [2,4, 6, 10],
        'l2_leaf_reg': [1, 3, 5]}

In [ ]:
model_final = model.randomized_search(grid,
                                      X=x,
                                      y=y,
                                      plot=True)

In [ ]:
submission.head()

In [ ]:
submission['Cover_Type'] = model_final.predict(test)
submission.to_csv('modelo_4.csv')